In [8]:
import pandas as pd
import numpy as np
from statsmodels.regression.linear_model import OLS
from statsmodels.discrete.discrete_model import Logit
import matplotlib.pyplot as plt

# Data collection
## Params and dates studied

In [9]:
params = ['HR_AVG_1.5m','TA_AVG_0.1m','PP_SUM_1.5m']
begin = '01/05/2022'
end = '01/09/2022'

In [10]:
DF = pd.read_json('https://servizos.meteogalicia.gal/mgrss/observacion/datosMensuaisEstacionsMeteo.action?idParam='+params[0]+','+params[1]+','+params[2]+'&dataIni='+begin+'&dataFin='+end)

## Export XY coordinates of meteorological stations

In [11]:
pd.DataFrame(DF.iloc[0][0]['listaEstacions'])[['estacion','idEstacion','utmx','utmy']].to_csv('Estaciones_XY.csv')

## Create a DataFrame with values of each parameter

In [12]:
df_data = []

for k in range(len(DF)): #Iteration over 12 months
    for i in range(len(pd.DataFrame(DF.iloc[k][0]['listaEstacions'])['listaMedidas'])): #Iteration over 155 stations
        
        date = DF.iloc[k][0]['data'][:7]
        idEst = DF.iloc[k][0]['listaEstacions'][i]['idEstacion']
        nameEst = DF.iloc[k][0]['listaEstacions'][i]['estacion']
        
        if (len(pd.DataFrame(DF.iloc[k][0]['listaEstacions'])['listaMedidas'].iloc[i]) == 3): # Take only data of stations that have Temp, Prec  HR
            if ((pd.DataFrame(DF.iloc[k][0]['listaEstacions'])['listaMedidas'].iloc[i][1]['valor'] > 0) 
                & (pd.DataFrame(DF.iloc[k][0]['listaEstacions'])['listaMedidas'].iloc[i][2]['valor'] > -9990)
                & (pd.DataFrame(DF.iloc[k][0]['listaEstacions'])['listaMedidas'].iloc[i][0]['valor'] > 0)): #Take only data without measurement errors
                
                df_data.append([date, idEst, nameEst, 
                                pd.DataFrame(DF.iloc[k][0]['listaEstacions'])['listaMedidas'].iloc[i][0]['valor'], 
                                pd.DataFrame(DF.iloc[k][0]['listaEstacions'])['listaMedidas'].iloc[i][1]['valor'],
                                pd.DataFrame(DF.iloc[k][0]['listaEstacions'])['listaMedidas'].iloc[i][2]['valor']
                               ])
            
df_data = pd.DataFrame(df_data)
df_data.columns = ['date','id','name','RH_AVG_1.5','PP_SUM','TA_AVG_0.1m']

## Export aggregated parameters (RH and TA = aggregated by mean) (PP aggregated by sum)

In [13]:
df_data.groupby('id').mean().iloc[:,[0,2]].to_csv('RH_n_TA_AVG.csv')
prec = pd.DataFrame(df_data.groupby('id').sum().iloc[:,1])
prec['PP_SUM'] = prec['PP_SUM']/4
prec.to_csv('PP_sum.csv')

# Linear regression analysis

In [14]:
DF = pd.read_excel('BurneArea_RegressionData.xls').iloc[:,[2, 10, 15, 20, 25]].dropna()
DF.columns = ['BurnedArea','MeanTemp','MeanRH','MeanPrec','MeanSlope']
DF['Constant'] = np.ones(len(DF))

lin_model = OLS(DF['BurnedArea'], DF[['MeanTemp','MeanRH','MeanPrec','MeanSlope']]).fit()

lin_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             BurnedArea   R-squared (uncentered):                   0.491
Model:                            OLS   Adj. R-squared (uncentered):              0.468
Method:                 Least Squares   F-statistic:                              21.92
Date:                Thu, 06 Oct 2022   Prob (F-statistic):                    1.09e-12
Time:                        20:37:20   Log-Likelihood:                         -910.89
No. Observations:                  95   AIC:                                      1830.
Df Residuals:                      91   BIC:                                      1840.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
MeanTemp    -175.2356    199.264     -0.879      0.381    -571.049     220.578
MeanRH        65.9213     67.246      0.980      0.330     -67.654     199.496
MeanPrec     -23.0628      9.852     -2.341      0.021     -42.632      -3.493
MeanSlope    301.6451     45.059      6.695      0.000     212.142     391.148
==============================================================================
Omnibus:                       10.743   Durbin-Watson:                   2.409
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               10.880
Skew:                           0.746   Prob(JB):                      0.00434
Kurtosis:                       3.724   Cond. No.                         144.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""